In [111]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
import patsy

file_path_filled = "../dataset/movie_metadata_cleaned_filled_values.csv"
file_path_empty  = "../dataset/movie_metadata_cleaned_empty.csv"
dta = pd.read_csv(file_path_filled)
#dta_e = pd.read_csv(file_path_empty)
pd.options.display.max_colwidth = 50
pd.options.display.max_columns = 100

str_list = ['color', 'movie_title', 'director_name', 'actor_2_name', 'genres',
            'actor_1_name', 'actor_3_name', 'plot_keywords', 'language', 'country', 'content_rating']

def removeWh(val,extra = ""):
    if type(val) == str:   
        val = val.replace("|", " ").replace("-", " ").replace("-", " ").replace("'", " ").replace(".", " ").replace(" ", "").replace(" ", "").replace(" ", "").replace(" ", "").replace(" ", "")
        if extra != "": val = extra + "_" + val
    return val

def removeStrip(val):
    if type(val) == str: 
        val = val.replace("|", " ").replace("-", "").replace("-", "").replace("'", "").replace(".", "")
    return val

def genString(val, col):
    if type(val) == str: 
        val = ' ' + val
    else:
        val = ''
    return val
#prepare data
for colname, colvalue in dta.iteritems():
    if type(colvalue[1]) == str and colname != 'movie_imdb_link' and colname != 'release_date':
        if colname == 'director_name' or colname == 'actor_1_name' or colname == 'actor_2_name' or colname == 'actor_3_name':
            dta[colname] = dta[colname].apply(lambda val: removeWh(val))
        if colname == 'genres' or colname == 'plot_keywords':
            dta[colname] = dta[colname].apply(lambda val: removeStrip(val))
        if colname == 'country':
            dta[colname] = dta[colname].apply(lambda val: removeWh(val, "country"))
        if colname == 'language':
            dta[colname] = dta[colname].apply(lambda val: removeWh(val, "language"))
        if colname == 'color':
            dta[colname] = dta[colname].apply(lambda val: removeWh(val, "color"))
        if colname == 'content_rating':
            dta[colname] = dta[colname].apply(lambda val: removeWh(val, "raiting"))
            
#generate common string
dta['string'] = ''
dta['category'] = ''
for colname, colvalue in dta.iteritems():
    if type(colvalue[1]) == str and colname != 'movie_imdb_link' and colname != 'release_date' and colname != 'string' and colname !='category': 
        if colname == 'country' or colname == 'content_rating' or colname == 'language' or colname == 'color' :
            dta['category'] += dta[colname].apply(lambda val: genString(val, colname))
        else:
            dta['string'] += dta[colname].apply(lambda val: genString(val, colname))

pd.options.display.max_colwidth = 500
pd.DataFrame(dta['category']) 

,category
0,color_Color language_English country_USA raiting_PG13
1,color_Color language_English country_USA raiting_PG13
2,color_Color language_English country_USA raiting_PG13
3,color_Color language_English country_USA raiting_PG13
4,color_Color language_English country_USA raiting_PG13
5,color_Color language_English country_UK raiting_PG13
6,color_Color language_English country_USA raiting_PG
7,color_Color language_English country_USA raiting_PG13
8,color_Color language_English country_USA raiting_PG
9,color_Color language_English country_USA raiting_PG13


In [134]:

tf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=4, stop_words='english', norm='l2', analyzer = 'word')
X_tf = tf_vectorizer.fit_transform(dta['string'])
tf_fr = pd.DataFrame(X_tf.todense(), columns = tf_vectorizer.get_feature_names())

################################
cat_vectorizer = CountVectorizer(max_df=1.0,min_df=1, stop_words='english' , analyzer = 'word')
X_cat = cat_vectorizer.fit_transform(dta['category'])
cat_fr = pd.DataFrame(X_cat.todense(), columns = cat_vectorizer.get_feature_names())

##################
print("tf shape:")
print(tf_fr.shape)
print("cat len")
print(cat_fr.shape)

strings = []
for colname, colvalue in dta.iteritems():
    if type(colvalue[1]) == str:
        strings.append(colname)
#remove old gross, budget and aspect_ratio
strings.append('gross')
strings.append('budget')
strings.append('aspect_ratio')
num_list = dta.columns.difference(strings)
##remove strings values from original data set
dta_clean = dta[num_list]


result = pd.concat([dta_clean, cat_fr, tf_fr], axis=1)

result.dropna()

print("res shape:")
print (result.shape)

tf shape:
(4812, 3480)
cat len
(4812, 123)
res shape:
(4812, 3619)


In [135]:
result.to_csv(path_or_buf="../dataset/movie_metadata_cleaned_tfidf_num_only_min.csv")

In [136]:
_y, _X = patsy.dmatrices('worldwide_gross ~  -1 + color + country + language  + content_rating',
                 dta, return_type="dataframe")

In [137]:
tf_fr

,007,10,11,12,13,13th,15,17th,18th,1920s,1930s,1940s,1950s,1960s,1970s,1980s,1990s,19th,20,2000,2000s,2001,21,30,3d,50cent,51,_abernal,_guez,_n,_rez,aaronpaul,aaronyoo,aasifmandvi,abandoned,abbiecornish,abduction,abortion,abroad,absent,abuse,abusive,accent,accident,accidental,accountant,act,action,activist,activity,...,wizard,wolf,wolfgangpetersen,woman,womanizer,women,womens,woods,woodstock,woodyallen,word,words,work,workaholic,worker,working,world,worlds,wrath,wrestler,wrestling,writer,writing,written,wrong,wrongful,xanderberkeley,yakuza,yaphetkotto,yard,yayadacosta,year,years,yellow,york,yorkshire,young,younger,zacharygordon,zachgilford,zacksnyder,zackward,zero,zeus,zo,zoekazan,zombie,zone,zoo,zooeydeschanel
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.126534,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.281322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.087867,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.186098,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.253925,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.104696,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.099885,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.153280,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
6,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0